In [ ]:
import pandas as pd
import numpy as np
from privacypacking.utils.utils import LOGS_PATH
from experiments.ray.analysis import load_ray_experiment
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
import copy

In [ ]:
logs = LOGS_PATH.joinpath("ray/run_and_report_2022-02-02_22-07-33")
df = load_ray_experiment(logs)
df.keys()
df = df[['scheduler_metric', 'tasks']]

dfs = []
ds = None
for i in range(len(df)):
    row = df.iloc[i]
    tasks = pd.DataFrame(row['tasks'])
    tasks = tasks[tasks['allocated'] == True]
    tasks.sort_values(['allocation_index'])
    dominant_shares = {}
    scheduler = [row['scheduler_metric']]

    for j in range(len(tasks)):
        dominant_share_per_alpha = {}
        budget_per_block = tasks.iloc[j].budget_per_block
        for block_id in budget_per_block:
            for alpha, epsilon in budget_per_block[block_id]['orders'].items():

                if alpha not in dominant_share_per_alpha:                 
                    dominant_share_per_alpha[alpha] = epsilon
                elif dominant_share_per_alpha[alpha] < epsilon:
                    dominant_share_per_alpha[alpha] = epsilon
        
        for key,value in dominant_share_per_alpha.items():
            if key not in dominant_shares:
                dominant_shares[key] = []
            dominant_shares[key] += [value]

    ds = pd.DataFrame.from_dict(dominant_shares)
    ds['scheduler_metric'] = scheduler*len(tasks)
    ds['task_id'] = range(0, len(tasks))
    dfs.append(ds)
    

dfs = pd.concat(dfs)
# print(dfs)



 
## Fairness vs Efficienty


In [ ]:
px.line(dfs, x="task_id", y="3", color='scheduler_metric')